Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

- On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
- La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

Voici une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
import re
import requests
import ipaddress
import time

#API_KEY = '51cb1f41306fa6dbac8ceea2b142cdc6'

### 1. Chargement des données dans un dataframe

In [2]:
url = 'https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv'

df = pd.read_csv(url, sep=';')

In [3]:
# vérification du remplissage des colonnes
print(df.info())
# vérification des premières lignes
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
username      200 non-null object
ip_address    200 non-null object
product       200 non-null object
price         200 non-null object
infos         200 non-null object
dtypes: object(5)
memory usage: 7.9+ KB
None


,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


### 2. Prix unifiés en euros

- extraction de la currency de la colonne price lorsqu'elle est disponible
- pour les lignes sans indication de currency : déduction à partir de l'adresse IP, lorque l'adresse IP est valide
  - suppression des lignes pour lesquelles il n'y a pas d'indication de currency ni d'adresse IP valide 
  - appel de l'API geoplugin.net pour déduire la currency de l'adresse IP 
  - suppression des lignes pour lesquelles on n'a rien trouvé
- calcul du taux de change (appel d'API) pour toutes les currency trouvées
- création d'une colonne prix_en_euros et remplissage par la multiplication du prix par le taux de change correspondant

In [4]:
# extract currency from price and handle price as float
df['currency'] = df.price.str.split(' ').str[1]
df['price'] = pd.to_numeric(df.price.str.split(' ').str[0])

In [5]:
# check result
print('price : ' + str(df.price.dtype))
print('currency :' + str(df.currency.unique()))

price : float64
currency :[nan 'MGA' 'EUR' 'USD' 'CNY' 'SEK' 'ARS' 'KES' 'BRL' 'AMD' 'IDR' 'RUB']


In [6]:
# determine whether IP is valid or not

def valid_ip(address):
    try:
        print(ipaddress.ip_address(address))
        return True
    except:
        return False

df['valid_ip'] = df.apply(lambda row: valid_ip(row['ip_address']), axis=1)

df.head()


240.177.79.234
26.191.237.49
58.90.204.239
226.52.32.70
127.197.204.119
189.169.17.54
187.129.113.105
22.32.234.215
75.254.207.163
17.105.113.72
208.239.186.79
116.18.56.20
87.198.88.207
138.44.156.99
18.61.195.234
150.214.20.140
157.197.190.113
219.75.213.15
225.250.18.162
157.145.5.133
105.123.55.143
222.143.75.39
17.226.242.68
201.51.7.205
177.248.202.127
24.1.79.147
123.157.88.33
254.37.135.127
254.21.208.244
134.167.195.239
140.223.76.83
67.232.83.24
124.100.237.236
12.218.65.100
22.73.245.49
191.45.106.36
32.112.67.248
188.98.203.195
167.174.244.176
21.124.12.215
190.233.51.35
143.75.128.57
41.63.88.3
32.77.50.179
201.14.107.40
109.144.241.78
136.251.73.188
246.216.13.211
220.115.202.199
69.135.112.215
223.25.136.102
6.120.0.29
178.154.116.57
207.0.191.149
109.242.195.131
38.125.216.233
35.105.188.50
104.27.157.229
133.56.53.78
46.14.231.6
104.137.59.250
176.211.210.96
244.35.187.136
40.155.15.190
43.219.225.156
127.128.57.9
127.8.161.201
86.115.203.240
142.7.77.36
155.168.213.24

,username,ip_address,product,price,infos,currency,valid_ip
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,NaN,False
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,NaN,False
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,NaN,True
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,NaN,True
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",NaN,True


In [7]:
# keep only the lines with either defined currency or valid IP
# i.e. remove the 12 lines that have no currency and invalid IP

df = df[(df.valid_ip == True) | (pd.notna(df.currency))]

#check results

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 2 to 198
Data columns (total 7 columns):
username      188 non-null object
ip_address    188 non-null object
product       188 non-null object
price         188 non-null float64
infos         188 non-null object
currency      17 non-null object
valid_ip      188 non-null bool
dtypes: bool(1), float64(1), object(5)
memory usage: 10.5+ KB
None


,username,ip_address,product,price,infos,currency,valid_ip
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,NaN,True
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,NaN,True
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",NaN,True
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MGA,True
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,NaN,True


In [8]:
# retrieve currency from IP address (NaN if a problem occurs during retrieval)

# API selection :
# usercountry.com : discontinued, consider ipstack instead
# ipstack : currency not part of the free API
# geoplugin : free lookup limit of 120 requests per minute, then error '403 Forbidden', thus the sleep


def get_currency(ip_address):
    time.sleep(0.6)
    url = f'http://www.geoplugin.net/json.gp?ip={ip_address}'
    resp = requests.get(url)
    if resp.status_code != 200:
        print('OOPS : response KO '+ resp.content)
        currency = 'NaN'
    else:
        geo_info = resp.json()
        currency = geo_info['geoplugin_currencyCode']
        #    print(currency)
    return currency

# compute currency for all valid IP addresses

df.loc[df.valid_ip == True, 'computed_currency'] = df[df.valid_ip == True].apply(
    lambda row: get_currency(row['ip_address']), axis=1)



In [9]:
# complete the currency column with the computed one 

df['currency'] = df.currency.combine_first(df.computed_currency)

currencies = df.currency.unique()
print("Currencies :" + str(currencies))
print()
print("Currency still missing : ")

missing = df[pd.isna(df.currency)]
print(missing.shape)

# as we really don't know what the cost is for those ones, just drop them
df = df.dropna(subset=['currency'])
print()
print(df.info())

df.head()


Currencies :[None 'USD' 'JPY' 'MGA' 'MXN' 'CNY' 'EUR' 'AUD' 'KRW' 'NGN' 'BRL' 'PEN'
 'GBP' 'BYN' 'RUB' 'SEK' 'CAD' 'MYR' 'ARS' 'VND' 'KES' 'INR' 'TWD' 'DKK'
 'IDR' 'BGN' 'IRR' 'AOA' 'AMD' 'KZT' 'TND' 'RON']

Currency still missing : 
(29, 8)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 3 to 198
Data columns (total 8 columns):
username             159 non-null object
ip_address           159 non-null object
product              159 non-null object
price                159 non-null float64
infos                159 non-null object
currency             159 non-null object
valid_ip             159 non-null bool
computed_currency    156 non-null object
dtypes: bool(1), float64(1), object(6)
memory usage: 10.1+ KB
None


,username,ip_address,product,price,infos,currency,valid_ip,computed_currency
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,True,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,True,JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MGA,True,None
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,True,MXN
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN,True,MXN


In [11]:
# compute change for all currencies to euros

# API selection: 
# - geoplugin does return 0 for VND and IRR
# - api.exchangerate-api.com does return 0 for IRR and does not support MGA
# I did not yet succeed in finding an API that supports all currencies 

currencies = df.currency.unique()
print("currencies :" + str(currencies))

def get_change(base_currency):
    time.sleep(0.6)
    url = f'http://www.geoplugin.net/json.gp?base_currency={base_currency}'
    resp = requests.get(url)
    if resp.status_code != 200:
        print('oops')
        print(resp.content)
        change = 'NaN'
    else:
        geo_info = resp.json()
        change = geo_info['geoplugin_currencyConverter']
    return change

change = {}
for currency in currencies:
    change[currency] = get_change(currency)

print(change)


currencies :['USD' 'JPY' 'MGA' 'MXN' 'CNY' 'EUR' 'AUD' 'KRW' 'NGN' 'BRL' 'PEN' 'GBP'
 'BYN' 'RUB' 'SEK' 'CAD' 'MYR' 'ARS' 'VND' 'KES' 'INR' 'TWD' 'DKK' 'IDR'
 'BGN' 'IRR' 'AOA' 'AMD' 'KZT' 'TND' 'RON']
{'USD': 0.8964, 'JPY': 0.0083, 'MGA': 0.0002, 'MXN': 0.0469, 'CNY': 0.1267, 'EUR': 1, 'AUD': 0.6166, 'KRW': 0.0008, 'NGN': 0.0025, 'BRL': 0.2171, 'PEN': 0.2683, 'GBP': 1.164, 'BYN': 0.4394, 'RUB': 0.0141, 'SEK': 0.093, 'CAD': 0.6853, 'MYR': 0.2142, 'ARS': 0.0154, 'VND': 0, 'KES': 0.0086, 'INR': 0.0126, 'TWD': 0.0293, 'DKK': 0.1338, 'IDR': 0.0001, 'BGN': 0.5112, 'IRR': 0, 'AOA': 0.002, 'AMD': 0.0019, 'KZT': 0.0023, 'TND': 0.3175, 'RON': 0.2101}


In [12]:
df['price_in_euros'] = df.apply(
    lambda row: row['price'] * change[row['currency']], axis=1)

del df['valid_ip']
del df['computed_currency']

#print(df.info())
df.head()

,username,ip_address,product,price,infos,currency,price_in_euros
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,313.874460
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.883257
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MGA,0.070866
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,5.232164
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN,4.129545


### 3. Ingrédients

- Extraction des ingrédients indiqué dans la colonne infos

- Stockage de la présence (potentielle) de chaque ingrédient sous la forme d'un booléen par colonne dédiée.

- Suppression de la colonne infos

In [13]:
#extract ingredients from the 'infos' column

ingredients = df.infos.str.get_dummies(sep=' ')
#ingredients.info()

In [14]:
# put the ingredients individually back as boolean

df['egg']=ingredients['egg'].astype(bool)
df['fish']=ingredients['fish'].astype(bool)
df['gluten']=(ingredients['gluten']+ingredients['gluten,']).astype(bool)
df['milk']=ingredients['milk'].astype(bool)
df['mustard']=(ingredients['mustard']+ingredients['mustard,']).astype(bool)
df['peanut']=(ingredients['peanut']+ingredients['peanut,']).astype(bool)
df['soja']=(ingredients['soja']+ingredients['soja,']).astype(bool)
df['sugar']=(ingredients['sugar']+ingredients['sugar,']).astype(bool)

# remove the now useless 'infos' column

del df['infos']

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 3 to 198
Data columns (total 14 columns):
username          159 non-null object
ip_address        159 non-null object
product           159 non-null object
price             159 non-null float64
currency          159 non-null object
price_in_euros    159 non-null float64
egg               159 non-null bool
fish              159 non-null bool
gluten            159 non-null bool
milk              159 non-null bool
mustard           159 non-null bool
peanut            159 non-null bool
soja              159 non-null bool
sugar             159 non-null bool
dtypes: bool(8), float64(2), object(4)
memory usage: 9.9+ KB


In [16]:
df.head()

,username,ip_address,product,price,currency,price_in_euros,egg,fish,gluten,milk,mustard,peanut,soja,sugar
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,USD,313.874460,False,False,True,False,False,False,False,False
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,JPY,7.883257,True,True,False,False,False,False,False,True
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,MGA,0.070866,False,False,False,True,False,False,False,True
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,MXN,5.232164,False,False,False,False,False,False,False,True
8,epridham8,187.129.113.105,Dried Figs,88.05,MXN,4.129545,False,True,False,True,False,False,False,True
